CODE
Подход 2: логистическая регрессия

In [2]:
# Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! 
# Может пригодиться sklearn.preprocessing.StandartScaler.

import numpy as np
import pandas
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold,cross_val_score

features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
y = features['radiant_win'].to_frame()
features.drop(['duration',
               'radiant_win',
               'tower_status_radiant', 
               'tower_status_dire', 
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)
features_clean = features.fillna(0)
features_test_clean = features_test.fillna(0)

rescale_attr = StandardScaler()
features_fin = rescale_attr.fit_transform(features_clean)
Y=np.array(y)


kf = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

In [3]:
#1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с 
# L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для 
# градиентного бустинга. 
from sklearn.linear_model import LogisticRegression

lr_qual = []
def run_LogReg(features_fin,Y):
    for i in range(-5,5):
        C=10**i
        lr = LogisticRegression(C=C, random_state=241)
        start_time = datetime.datetime.now()
        lr_scores = cross_val_score(lr, features_fin, Y.ravel(), cv=kf, scoring='roc_auc')
        run_time=datetime.datetime.now() - start_time
        print ('Time elapsed:', run_time)
        print (lr_scores)
        lr_qual.append((i,C,run_time.total_seconds(),np.mean(lr_scores)))
    return lr_qual

In [4]:
run_LogReg(features_fin, Y)
lr_qual

Time elapsed: 0:00:03.458511
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
Time elapsed: 0:00:04.726167
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
Time elapsed: 0:00:08.460083
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
Time elapsed: 0:00:11.018461
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
Time elapsed: 0:00:10.981462
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
Time elapsed: 0:00:12.264499
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
Time elapsed: 0:00:12.672545
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
Time elapsed: 0:00:11.919500
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
Time elapsed: 0:00:11.903844
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
Time elapsed: 0:00:11.665508
[0.72165733 0.71369457 0.71438593 0.71539407 0.71639938]


[(-5, 1e-05, 3.458511, 0.695120379847076),
 (-4, 0.0001, 4.726167, 0.7112501143920594),
 (-3, 0.001, 8.460083, 0.7161802463683578),
 (-2, 0.01, 11.018461, 0.716341462186996),
 (-1, 0.1, 10.981462, 0.7163100836533356),
 (0, 1, 12.264499, 0.716306583645544),
 (1, 10, 12.672545, 0.7163063399602339),
 (2, 100, 11.9195, 0.7163062657792337),
 (3, 1000, 11.903844, 0.7163062636530346),
 (4, 10000, 11.665508, 0.7163062572998156)]

In [5]:
# Подберите при этом лучший параметр регуляризации (C).
def bestC(lr_qual):
    lr_qual.sort(key=lambda x:x[3])
    return max(lr_qual, key=lambda x: x[3])[3]

In [6]:
# Какое наилучшее качество у вас получилось?
all_values = bestC(lr_qual)
all_values

0.716341462186996

 Как оно соотносится с качеством градиентного бустинга? 
 
 0.716341462186996 -> similar to GB with ntrees=200


In [7]:
# Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
time_required = max(lr_qual, key=lambda x: x[3])[2]
time_required #=10.079412

11.018461

GB: datetime.timedelta(0, 321, 256976).total_seconds() = 321.256976

->  30 times faster

In [8]:
#2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, 
#что вряд ли является хорошей идеей.


In [9]:
#Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, 
#d2_hero, ..., d5_hero. Уберите их из выборки,

In [10]:
features_clean_nocat=features_clean.copy()

In [11]:
def remove_cat_attr(features_nocat):
    del features_nocat['lobby_type']
    for n in range(1, 6):
        del features_nocat['r{}_hero'.format(n)]
        del features_nocat['d{}_hero'.format(n)]
    return features_nocat

In [12]:
remove_cat_attr(features_clean_nocat)
features_clean_nocat[:2]

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0


In [13]:
features_fin_nocat = rescale_attr.fit_transform(features_clean_nocat)

In [14]:
run_LogReg(features_fin_nocat, Y)



Time elapsed: 0:00:02.895299
[0.69961316 0.69150564 0.69419646 0.69425458 0.69571482]
Time elapsed: 0:00:03.953975
[0.71630684 0.70828717 0.70971489 0.71020968 0.71172337]
Time elapsed: 0:00:07.105742
[0.72152794 0.71360976 0.71445497 0.71521698 0.71636831]
Time elapsed: 0:00:09.178326
[0.7218164  0.71370124 0.71462628 0.71551614 0.71634469]
Time elapsed: 0:00:09.647376
[0.72181205 0.7136378  0.7146095  0.71551012 0.71629945]
Time elapsed: 0:00:10.145465
[0.72181039 0.71363045 0.71460884 0.71551016 0.71629392]
Time elapsed: 0:00:09.785907
[0.72181043 0.71362965 0.71460893 0.71551033 0.71629306]
Time elapsed: 0:00:10.433436
[0.72181067 0.71362945 0.71460889 0.71551027 0.71629322]
Time elapsed: 0:00:09.661108
[0.72181072 0.71362952 0.71460893 0.71551026 0.71629323]
Time elapsed: 0:00:09.771569
[0.72181071 0.71362952 0.71460894 0.71551026 0.71629323]


[(-5, 1e-05, 3.458511, 0.695120379847076),
 (-4, 0.0001, 4.726167, 0.7112501143920594),
 (-3, 0.001, 8.460083, 0.7161802463683578),
 (4, 10000, 11.665508, 0.7163062572998156),
 (3, 1000, 11.903844, 0.7163062636530346),
 (2, 100, 11.9195, 0.7163062657792337),
 (1, 10, 12.672545, 0.7163063399602339),
 (0, 1, 12.264499, 0.716306583645544),
 (-1, 0.1, 10.981462, 0.7163100836533356),
 (-2, 0.01, 11.018461, 0.716341462186996),
 (-5, 1e-05, 2.895299, 0.6950569329910983),
 (-4, 0.0001, 3.953975, 0.7112483906159717),
 (-3, 0.001, 7.105742, 0.7162355910206267),
 (-2, 0.01, 9.178326, 0.7164009506527343),
 (-1, 0.1, 9.647376, 0.7163737844721112),
 (0, 1, 10.145465, 0.7163707526581122),
 (1, 10, 9.785907, 0.7163704793048005),
 (2, 100, 10.433436, 0.7163704962706654),
 (3, 1000, 9.661108, 0.7163705301659756),
 (4, 10000, 9.771569, 0.7163705301674704)]

In [15]:
# и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
nocat_values = bestC(lr_qual)
nocat_values

0.7164009506527343

In [16]:
nocat_values/all_values

1.0000830448450613

 Изменилось ли качество? - no, same
 
Чем вы можете это объяснить? - those attributes do not have any impact on the model

In [17]:
#На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли 
# за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, 
# чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может 
# пригодиться фукнция unique или value_counts).

In [ ]:
features_heros=features_clean.copy()
all_heroes1=[]
all_heroes2=[]
for n in range(1, 6):
    all_heroes1.append(features_heros['r{}_hero'.format(n)].unique())
    all_heroes2.append(features_heros['d{}_hero'.format(n)].unique())


In [ ]:
features_heros['r1_hero'][2]

In [ ]:
all_names_of_heroes=[]
for i in range(0,5):
    for j in range(0,len(all_heroes2[i])):
        all_names_of_heroes.append(all_heroes2[i][j])
for i in range(0,5):
    for j in range(0,len(all_heroes1[i])):
        all_names_of_heroes.append(all_heroes1[i][j])        

In [ ]:
hero_names = list(set(all_names_of_heroes))

In [ ]:
Num_heroes = len(hero_names)
print (Num_heroes)

In [ ]:
#4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев.
#Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, 
#если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. 
#Ниже вы можете найти код, который выполняет данной преобразование. 

# N — количество различных героев в выборке
# Формируем "мешок слов" по героям
def bag_hero_to_num(features):
    X_pick = np.zeros((features.shape[0], 112))
    for i, match_id in enumerate(features.index):
        for p in range(0,5):
            X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    return pandas.DataFrame(X_pick, index=features.index)

In [ ]:
X_pick

In [ ]:
bag_hero_to_num(features)

In [ ]:
hero_pd = pandas.DataFrame(X_pick, index=features.index)

In [ ]:
hero_pd[:8]

In [ ]:
#Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
features_clean_catToNum_base = features_clean.copy()

In [ ]:
features_clean_catToNum = pandas.concat([features_clean_catToNum_base, hero_pd], axis=1)

In [ ]:
features_fin_catToNum = rescale_attr.fit_transform(features_clean_catToNum)
#features_fin_catToNum[:2]

In [ ]:
#4 .Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
run_LogReg(features_fin_catToNum, Y)

In [ ]:
values_with_catToNum = bestC(lr_qual)
values_with_catToNum


In [ ]:
values_with_catToNum/nocat_values

Какое получилось качество? 
0.7519018100855892

Улучшилось ли оно?
+5%

Чем вы можете это объяснить?

Proper use of all atributes: use category attribute as information - not remove, not directly digital

In [ ]:
#Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей
#(лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке 
# [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [ ]:
C=0.01
lr = LogisticRegression(C=C, random_state=241)
lr.fit(features_fin_catToNum, Y.ravel())

In [ ]:
features_test_clean_nocat = features_test_clean.copy()
remove_cat_attr(features_test_clean_nocat)

hero_pd_test = bag_hero_to_num(features_test)

features_test_clean_catToNum = pandas.concat([features_test_clean_nocat, hero_pd_test], axis=1)

features_test_fin1 = rescale_attr.fit_transform(features_test_clean_catToNum)



In [ ]:
lr.predict_proba(features_test_fin1)[:,1]